# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [25]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import cv2

We set the random seed so that the results don't vary drastically.

In [26]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

### All input Data keeping in one place for the Generator:

In [27]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10 # Increase the batch size and check the performance of GPU
n_frame = 30 # number of frames
y = 120 # image width
z = 120 # image height
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop 
channel = 3
curr_dt_time = datetime.datetime.now()
source_path=train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


# training sequences = 663
# validation sequences = 100
# epochs = 10


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [28]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,n_frame)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,n_frame,y,z,3)) # n_frame is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    tempImage = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)              
                    
                    batch_data[folder,idx,:,:,0] = (tempImage[:,:,0])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (tempImage[:,:,1])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (tempImage[:,:,2])/255.0 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            #print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,n_frame,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                #print("batch:",batch,num_batches)
                # Replace batch with 'num_batches'
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    tempImage = cv2.resize(image, (y,z), interpolation = cv2.INTER_AREA)          
                    
                    batch_data[folder,idx,:,:,0] = (tempImage[:,:,0])/255.0
                    batch_data[folder,idx,:,:,1] = (tempImage[:,:,1])/255.0
                    batch_data[folder,idx,:,:,2] = (tempImage[:,:,2])/255.0
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels         


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model 1
- Here you make the model using different functionalities that Keras provides. 
- CNN+RNN Architecture
- We will use the concept of transfer Learning using the base model as VGG16

In [29]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D,Conv3D, MaxPooling3D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras




In [30]:
#write your model here
nb_featuremap = [8,16,32,64,128]
#nb_dense = [128,64,5]
nb_dense = [128,64]
nb_classes = 5
# Input
input_shape=(n_frame,y,z,3)

model = Sequential()

model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add((Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add((Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


#model.add(TimeDistributed(Conv2D(nb_featuremap[4], (2,2),padding='same')))
#model.add(TimeDistributed(BatchNormalization()))
#model.add(TimeDistributed(Activation('relu')))
#model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

#model.add(TimeDistributed(BatchNormalization()))
#model.add(Dropout(0.5))

model.add(TimeDistributed(Flatten()))

model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


#model.add(Dense(nb_dense[0], activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(nb_dense[1], activation='relu'))
#model.add(Dropout(0.5))


## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(nb_classes, activation='softmax')) # using Softmax as last layer

In [31]:
 #write your optimizer
learning_rate=0.001
optimiser = optimizers.Adam(learning_rate) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 60, 60, 16)    64        
_________________________________________________________________
activation_12 (Activation)   (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 30, 30, 30, 32)    128       
__________

In [32]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

#LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [ ]:
batch_size = 10 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 10
<generator object generator at 0x7f50bf84b728>
<generator object generator at 0x7f50bf84b570>
Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10
10/67 [===>..........................] - ETA: 2:24 - loss: 1.5147 - categorical_accuracy: 0.3200

#### Experiment 2 : Batch Size increased to 20

In [10]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 10
<generator object generator at 0x7f5a3b2c5ca8>
<generator object generator at 0x7f5a3b2c55c8>
Source path =  Project_data/val ; batch size =Source path =  Project_data/train ; batch size = 20
Epoch 1/10
 20
32/34 [===========================>..] - ETA: 1s - loss: 0.5110 - categorical_accuracy: 0.8219Batch:  34 Index: 20
batch: 32 33
batch: 32 33
batch: 32 33
34/34 [==============================] - 25s 749ms/step - loss: 0.5397 - categorical_accuracy: 0.8043 - val_loss: 1.0229 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2020-06-2808_47_22.999929/model-00001-0.51960-0.81599-1.02291-0.65000.h5
Epoch 2/10
34/34 [==============================] - 4s 129ms/step - loss: 0.8609 - categorical_accuracy: 0.7157 - val_loss: 1.0434 - val_categorical_accuracy: 0.6300

Epoch 00002: saving model to model_init_2020-06-2808_47_22.999929/model-00002-0.86090-0.71569-1.04344-0.63000.h5
Epoch 3/10
34/34 [==============================] - 4s 130ms/step - loss: 0

#### Experiment 3: Batch Size increased to 30

In [11]:
batch_size = 30 # Increase the batch size and check the performance of GPU
num_epochs = 10 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 10
<generator object generator at 0x7f5a3b2c5d58>
<generator object generator at 0x7f5ad8559e08>
Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/10
 3/23 [==>...........................] - ETA: 24s - loss: 0.3487 - categorical_accuracy: 0.8778Batch:  4 Index: 30
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
21/23 [==========================>...] - ETA: 2s - loss: 0.3896 - categorical_accuracy: 0.8698Batch:  23 Index: 30
batch: 21 22
batch: 21 22
batch: 21 22
23/23 [==============================] - 25s 1s/step - loss: 0.4860 - categorical_accuracy: 0.8304 - val_loss: 1.1585 - val_categorical_accuracy: 0.6200

Epoch 00001: saving model to model_init_2020-06-2808_47_22.999929/model-00001-0.40477-0.86275-1.15855-0.62000.h5
Epoch 2/10
23/23 [==============================] - 2s 96ms/step - loss: 0.8165 - categorical_accuracy: 0.6957 - val_loss: 1.0851 - v

#### Experiment 4 : Batch Size 30, 20 Epochs

In [12]:
batch_size = 30 # Increase the batch size and check the performance of GPU
num_epochs = 20 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 20
<generator object generator at 0x7f59f2276f10>
<generator object generator at 0x7f5a3b2c5d58>
Source path =  Project_data/val ; batch size = Source path =  30
Project_data/train ; batch size = 30
Epoch 1/20
 2/23 [=>............................] - ETA: 29s - loss: 0.3248 - categorical_accuracy: 0.9667Batch:  4 Index: 30
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
 3/23 [==>...........................] - ETA: 26s - loss: 0.3451 - categorical_accuracy: 0.9333batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
21/23 [==========================>...] - ETA: 2s - loss: 0.3874 - categorical_accuracy: 0.8937Batch:  23 Index: 30
batch: 21 22
batch: 21 22
batch: 21 22
23/23 [==============================] - 25s 1s/step - loss: 0.5027 - categorical_accuracy: 0.8536 - val_loss: 1.3121 - val_categorical_accuracy: 0.6400

Epoch 00001: saving model to model_init_2020-06-2808_47_22.999929/model-00001-0.40591-0.88688-1.31212-0.64000.h5
Epoch 2/20
23/23 [====================

#### Experiment 5 : Batch Size 30, 30 Epochs

In [13]:
batch_size = 30 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f59f0178360>
<generator object generator at 0x7f59f2276f10>
Source path =  Project_data/val ; batch size = 30
Source path = Epoch 1/30
 Project_data/train ; batch size = 30
 3/23 [==>...........................] - ETA: 24s - loss: 0.3955 - categorical_accuracy: 0.8778Batch:  4 Index: 30
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
batch: 2 3
21/23 [==========================>...] - ETA: 2s - loss: 0.3816 - categorical_accuracy: 0.8873Batch:  23 Index: 30
batch: 21 22
batch: 21 22
batch: 21 22
23/23 [==============================] - 25s 1s/step - loss: 0.4448 - categorical_accuracy: 0.8632 - val_loss: 1.1262 - val_categorical_accuracy: 0.6600

Epoch 00001: saving model to model_init_2020-06-2808_47_22.999929/model-00001-0.38754-0.88386-1.12622-0.66000.h5
Epoch 2/30
23/23 [==============================] - 3s 111ms/step - loss: 0.7597 - categorical_accuracy: 0.6957 - val_loss: 1.1462 - 

#### Experiment 6 : Batch Size 20, 20 Epochs

In [14]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 20 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 20
<generator object generator at 0x7f59f0178258>
<generator object generator at 0x7f59e9e3fc50>
Source path =  Project_data/val ; batch size =Source path =  Project_data/train ; batch size = 20
 20
Epoch 1/20
32/34 [===========================>..] - ETA: 1s - loss: 0.4143 - categorical_accuracy: 0.8656Batch:  34 Index: 20
batch: 32 33
batch: 32 33
batch: 32 33
34/34 [==============================] - 25s 740ms/step - loss: 0.4863 - categorical_accuracy: 0.8403 - val_loss: 0.9215 - val_categorical_accuracy: 0.6900

Epoch 00001: saving model to model_init_2020-06-2808_47_22.999929/model-00001-0.42533-0.86124-0.92151-0.69000.h5
Epoch 2/20
34/34 [==============================] - 4s 129ms/step - loss: 0.7284 - categorical_accuracy: 0.6765 - val_loss: 0.9210 - val_categorical_accuracy: 0.6900

Epoch 00002: saving model to model_init_2020-06-2808_47_22.999929/model-00002-0.72838-0.67647-0.92101-0.69000.h5
Epoch 3/20
34/34 [==============================] - 5s 134ms/step - loss: 0

#### Experiment 7 : Batch Size 10, 20 Epochs

In [15]:
batch_size = 10 # Increase the batch size and check the performance of GPU
num_epochs = 20 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 20
<generator object generator at 0x7f59f0178728>
<generator object generator at 0x7f59f21a0b48>
Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/trainEpoch 1/20
 ; batch size = 10
65/67 [============================>.] - ETA: 0s - loss: 0.4426 - categorical_accuracy: 0.8523Batch:  67 Index: 10
batch: 65 66
batch: 65 66
batch: 65 66
67/67 [==============================] - 25s 375ms/step - loss: 0.4598 - categorical_accuracy: 0.8454 - val_loss: 0.9204 - val_categorical_accuracy: 0.6800

Epoch 00001: saving model to model_init_2020-06-2808_47_22.999929/model-00001-0.44871-0.85068-0.92037-0.68000.h5
Epoch 2/20
67/67 [==============================] - 8s 126ms/step - loss: 0.7402 - categorical_accuracy: 0.7313 - val_loss: 0.9269 - val_categorical_accuracy: 0.6700

Epoch 00002: saving model to model_init_2020-06-2808_47_22.999929/model-00002-0.74025-0.73134-0.92686-0.67000.h5
Epoch 3/20
67/67 [==============================] - 8s 120ms/step - loss: 0

#### Experiment 8 : Batch Size 10, 30 Epochs

In [35]:
batch_size = 10 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f59e9aecca8>
<generator object generator at 0x7f59e9aecc50>
Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = Epoch 1/30
10
65/67 [============================>.] - ETA: 0s - loss: 1.2819 - categorical_accuracy: 0.4738Batch:  67 Index: 10
batch: 65 66
batch: 65 66
batch: 65 66
67/67 [==============================] - 27s 402ms/step - loss: 1.2905 - categorical_accuracy: 0.4692 - val_loss: 1.9789 - val_categorical_accuracy: 0.4100

Epoch 00001: saving model to model_init_2020-06-2809_11_30.882988/model-00001-1.28685-0.47059-1.97886-0.41000.h5
Epoch 2/30
67/67 [==============================] - 7s 110ms/step - loss: 1.3320 - categorical_accuracy: 0.4478 - val_loss: 1.8883 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to model_init_2020-06-2809_11_30.882988/model-00002-1.33197-0.44776-1.88831-0.39000.h5
Epoch 3/30
67/67 [==============================] - 9s 129ms/step - loss: 1

#### Experimt Batch size 40, epochs=30

In [36]:
batch_size = 40 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f59e8c856d0>
<generator object generator at 0x7f59e9aecca8>
Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/30
 2/17 [==>...........................] - ETA: 23s - loss: 0.2393 - categorical_accuracy: 0.9250Batch:  3 Index: 40
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
batch: 1 2
15/17 [=========================>....] - ETA: 2s - loss: 0.2405 - categorical_accuracy: 0.9383Batch:  17 Index: 40
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
16/17 [===========================>..] - ETA: 1s - loss: 0.2368 - categorical_accuracy: 0.9391batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
batch: 15 16
b

In [46]:
batch_size = 20 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f59dbc7cc50>
<generator object generator at 0x7f59db33ac50>
Source path =  Project_data/val ; batch size = Source path =  Project_data/train ; batch size = 20
Epoch 1/30
20
34/34 [==============================] - 26s 769ms/step - loss: 1.3310 - categorical_accuracy: 0.4533 - val_loss: 1.5455 - val_categorical_accuracy: 0.5400

Epoch 00001: saving model to model_init_2020-06-2809_23_36.935131/model-00001-1.28285-0.46456-1.54545-0.54000.h5
Epoch 2/30
34/34 [==============================] - 3s 103ms/step - loss: 1.3998 - categorical_accuracy: 0.3824 - val_loss: 1.8491 - val_categorical_accuracy: 0.4100

Epoch 00002: saving model to model_init_2020-06-2809_23_36.935131/model-00002-1.39984-0.38235-1.84911-0.41000.h5
Epoch 3/30
34/34 [==============================] - 4s 126ms/step - loss: 1.4553 - categorical_accuracy: 0.4020 - val_loss: 1.9129 - val_categorical_accuracy: 0.3900

Epoch 00003: saving model to model_init_2020-06-2809_23_36.935

In [47]:
batch_size = 60 # Increase the batch size and check the performance of GPU
num_epochs = 30 # choose the number of epochs and do the experiment by increasing the epochs
print ('# epochs =', num_epochs)

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# epochs = 30
<generator object generator at 0x7f59d9c60db0>
<generator object generator at 0x7f59f2276f10>
Source path =  Project_data/val ; batch size = 60
Source path =  Project_data/train ; batch size = 60
Epoch 1/30
12/12 [==============================] - 25s 2s/step - loss: 0.6838 - categorical_accuracy: 0.7625 - val_loss: 2.0635 - val_categorical_accuracy: 0.4900

Epoch 00001: saving model to model_init_2020-06-2809_23_36.935131/model-00001-0.57377-0.79638-2.06345-0.49000.h5
Epoch 2/30
12/12 [==============================] - 1s 121ms/step - loss: 0.8367 - categorical_accuracy: 0.6667 - val_loss: 0.9461 - val_categorical_accuracy: 0.6750

Epoch 00002: saving model to model_init_2020-06-2809_23_36.935131/model-00002-0.83667-0.66667-0.94605-0.67500.h5
Epoch 3/30
12/12 [==============================] - 1s 103ms/step - loss: 0.9069 - categorical_accuracy: 0.6111 - val_loss: 2.2764 - val_categorical_accuracy: 0.4500

Epoch 00003: saving model to model_init_2020-06-2809_23_36.935131

RuntimeError: Problems closing file (file write failed: time = Sun Jun 28 09:28:06 2020
, filename = 'model_init_2020-06-2809_23_36.935131/model-00028-1.01227-0.44444-0.90875-0.70000.h5', file descriptor = 62, errno = 28, error message = 'No space left on device', buf = 0x55888175e4d0, total write size = 7976, bytes this sub-write = 7976, bytes actually written = 18446744073709551615, offset = 4096)

# Gesture Recognition using CONV3D:

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [33]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))


model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))


model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))


model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))
model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())
model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
 #write your optimizer
learning_rate=0.01
optimiser = optimizers.Adam(learning_rate) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_4 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator at 0x7f1c5e6c4db0>
<generator object generator at 0x7f1c5d12bca8>


In [34]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - 53s 797ms/step - loss: 12.6587 - categorical_accuracy: 0.2079 - val_loss: 13.2168 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-06-2901_55_31.265860/model-00001-12.67859-0.20664-13.21684-0.18000.h5
Epoch 2/10
67/67 [==============================] - 14s 206ms/step - loss: 12.5096 - categorical_accuracy: 0.2239 - val_loss: 13.2168 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2020-06-2901_55_31.265860/model-00002-12.50957-0.22388-13.21684-0.18000.h5
Epoch 3/10
67/67 [==============================] - 16s 237ms/step - loss: 13.2313 - categorical_accuracy: 0.1791 - val_loss: 13.2168 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to model_init_2020-06-2901_55_31.265860/model-00003-13.23127-0.17910-13.21684-0.18000.h5

Epoch 00003: ReduceLROnPlateau reducing le

In [13]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
# Define model b
model_b = Sequential()
model_b.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=(n_frame,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))
model_b.add(Flatten())

#model_b.add(Dense(256, activation='relu'))
#model_b.add(Dropout(0.5))

model_b.add(Dense(128,activation='relu'))
model_b.add(BatchNormalization())
model_b.add(Dropout(0.5))

model_b.add(Dense(64,activation='relu'))
model_b.add(BatchNormalization())
model_b.add(Dropout(0.25))

model_b.add(Dense(classes, activation='softmax'))



model_b.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_b.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  13856     
_________________________________________________________________
activation_9 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [36]:
n_frame = 30 # number of frames
y = 120 # image width
z = 120 # image height
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop 
channel = 3
def generator_new(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size) 
    img_idx = [x for x in range(0,n_frame)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,n_frame,y,z,channel)) # n_frame is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                                                            
                    tempImage = imresize(image,(120,120))   
                    #Converting to gray scale
                    tempImage = tempImage.mean(axis=-1,keepdims=1) 
                    #Normalize data
                    tempImage=tempImage/127.5-1
                    batch_data[folder,idx]=tempImage #normalise and feed in the image                   
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):       
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,n_frame,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size               
                # Replace batch with 'num_batches'
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    tempImage = imresize(image,(120,120))   
                     #Converting to gray scale
                    tempImage = tempImage.mean(axis=-1,keepdims=1) 
                    #Normalize data                 
                    tempImage=tempImage/127.5-1
                    batch_data[folder,idx]=tempImage #normalise and feed in the image                
                   
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels         


### Experiment:1 on Batch Size =20

In [37]:
batch_size=20

In [38]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)

In [24]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train Epoch 1/10
; batch size = 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


31/34 [==========================>...] - ETA: 7s - loss: 1.9685 - categorical_accuracy: 0.2694 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 89s 3s/step - loss: 1.9598 - categorical_accuracy: 0.2715 - val_loss: 2.4822 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-06-2902_13_29.851005/model-00001-1.95231-0.26998-2.48219-0.31000.h5
Epoch 2/10
34/34 [==============================] - 16s 478ms/step - loss: 1.8808 - categorical_accuracy: 0.3431 - val_loss: 2.2958 - val_categorical_accuracy: 0.3900

Epoch 00002: saving model to model_init_2020-06-2902_13_29.851005/model-00002-1.88080-0.34314-2.29585-0.39000.h5
Epoch 3/10
34/34 [==============================] - 16s 474ms/step - loss: 1.8703 - categorical_accuracy: 0.2843 - val_loss: 3.6670 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-06-2902_13_29.851005/model-00003-1.87034-0.28431-3.66704-0.30000.h5
Epoch 4/10
34/34 [==============================] - 16s 473ms/step - loss: 2.2829 - categorical_accuracy: 0.1471 - val_loss: 3.1593 - val_categorical_accuracy: 0.2700

Epo

### Experiment:2 on Batch Size =30

In [25]:
batch_size=30
train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)

In [26]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [27]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 97s 4s/step - loss: 1.9566 - categorical_accuracy: 0.1931 - val_loss: 1.6023 - val_categorical_accuracy: 0.1900

Epoch 00001: saving model to model_init_2020-06-2902_13_29.851005/model-00001-1.94237-0.20060-1.60235-0.19000.h5
Epoch 2/10
23/23 [==============================] - 10s 427ms/step - loss: 2.0355 - categorical_accuracy: 0.1304 - val_loss: 1.5921 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-06-2902_13_29.851005/model-00002-2.03553-0.13043-1.59214-0.20000.h5
Epoch 3/10
23/23 [==============================] - 10s 430ms/step - loss: 1.8764 - categorical_accuracy: 0.2754 - val_loss: 1.6218 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-06-2902_13_29.851005/model-00003-1.87643-0.27536-1.62177-0.20000.h5
Epoch 4/10
23/23 [==============================] - 10s 426ms/step - loss: 1.7441 - categorical_accuracy: 0.2319 - val_loss: 1.6503 - val_categorical_accuracy: 0.1750

Epo

In [39]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
#nb_dense = [1000, 500, 5]
nb_dense = [256, 128, 5]

nb_frames=30
nb_rows=120
nb_cols=120
nb_channel =3

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model_c = Sequential()

model_c.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_c.add(BatchNormalization())
model_c.add(Activation('relu'))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

model_c.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_c.add(Activation('relu'))
model_c.add(Dropout(0.25))

model_c.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_c.add(Flatten())

model_c.add(Dense(nb_dense[0], activation='relu'))
model_c.add(Dropout(0.5))

model_c.add(Dense(nb_dense[1], activation='relu'))
model_c.add(Dropout(0.5))

#softmax layer
model_c.add(Dense(nb_dense[2], activation='softmax'))


learning_rate=0.001
optimiser = optimizers.Adam(learning_rate) #write your optimizer
model_c.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_c.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_22 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_19 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_23 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_20 (Activation)   (None, 15, 60, 60, 16)    0         
__________

### Experiment:3 on batch size =40

In [29]:
batch_size=40
print ('# epochs =', num_epochs)
print ('# batch_size =', batch_size)

# epochs = 10
# batch_size = 40


In [30]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [31]:
train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)

In [32]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/17 [>.............................] - ETA: 2:55 - loss: 4.5557 - categorical_accuracy: 0.2000

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 35s 2s/step - loss: 13.1739 - categorical_accuracy: 0.1827 - val_loss: 13.4317 - val_categorical_accuracy: 0.1667

Epoch 00004: saving model to model_init_2020-06-2902_13_29.851005/model-00004-13.17392-0.18266-13.43175-0.16667.h5
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 12.9634 - categorical_accuracy: 0.1957 - val_loss: 13.1631 - val_categorical_accuracy: 0.1833

Epoch 00005: saving model to model_init_2020-06-2902_13_29.851005/model-00005-12.97687-0.19489-13.16311-0.18333.h5

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 13.3853 - categorical_accuracy: 0.1696 - val_loss: 11.2827 - val_categorical_accuracy: 0.3000

Epoch 00006: saving model to model_init_2020-06-2902_13_29.851005/model-00006-13.38527-0.16955-11.28267-0.30000.h5
Epoch 7/10
17/17 [==============================] - 31s 2s/step - loss: 12.6602 - cat

### Experiment:3 on batch size =40

In [33]:
batch_size = 40
num_epochs = 15
print ('# epochs =', num_epochs)
print ('# batch_size =', batch_size)

# epochs = 15
# batch_size = 40


In [34]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [35]:

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator_new at 0x7fd65e54b4c0>
<generator object generator_new at 0x7fd65caf20f8>


In [36]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val Source path =  Project_data/train ; batch size = 40
; batch size = 40
Epoch 1/15


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 1:19 - loss: 12.2900 - categorical_accuracy: 0.2375

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 77s 5s/step - loss: 13.0632 - categorical_accuracy: 0.1895 - val_loss: 12.7333 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-06-2902_13_29.851005/model-00001-13.00631-0.19306-12.73330-0.21000.h5
Epoch 2/15
17/17 [==============================] - 41s 2s/step - loss: 12.4493 - categorical_accuracy: 0.2276 - val_loss: 12.6258 - val_categorical_accuracy: 0.2167

Epoch 00002: saving model to model_init_2020-06-2902_13_29.851005/model-00002-12.44927-0.22762-12.62584-0.21667.h5
Epoch 3/15
17/17 [==============================] - 42s 2s/step - loss: 13.2571 - categorical_accuracy: 0.1775 - val_loss: 12.0886 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-06-2902_13_29.851005/model-00003-13.30731-0.17439-12.08857-0.25000.h5
Epoch 4/15
17/17 [==============================] - 37s 2s/step - loss: 13.1240 - categorical_accuracy: 0.1858 - val_loss: 12.6258 - val_categorical_accuracy: 0.2167

### Experiment:4 on epoch size=20 and batch size =40

In [37]:
batch_size=40
num_epochs = 20 

In [38]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [39]:
train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator_new at 0x7fd6474dfeb8>
<generator object generator_new at 0x7fd65e54b4c0>


In [40]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train Epoch 1/20; batch size = 40



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 1:14 - loss: 12.4915 - categorical_accuracy: 0.2250

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 77s 5s/step - loss: 12.9827 - categorical_accuracy: 0.1945 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-06-2902_13_29.851005/model-00001-12.95768-0.19608-12.41093-0.23000.h5
Epoch 2/20
17/17 [==============================] - 43s 3s/step - loss: 12.8203 - categorical_accuracy: 0.2046 - val_loss: 12.8945 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-06-2902_13_29.851005/model-00002-12.82028-0.20460-12.89448-0.20000.h5
Epoch 3/20
17/17 [==============================] - 41s 2s/step - loss: 13.3887 - categorical_accuracy: 0.1693 - val_loss: 12.6258 - val_categorical_accuracy: 0.2167

Epoch 00003: saving model to model_init_2020-06-2902_13_29.851005/model-00003-13.35123-0.17166-12.62584-0.21667.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.9531249563442543e-05.
Epoch 4/20
17/17 [==============================] - 36s 2s/step - loss: 13.1240 - c

### Experiment:5 on epoch size=15 and batch size =30

In [40]:
batch_size=30
num_epochs = 10 

In [41]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [42]:
train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)
print(train_generator)
print(val_generator)

<generator object generator_new at 0x7f1c5d12bca8>
<generator object generator_new at 0x7f1c481eb518>


In [43]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train ; batch size = 30
Epoch 1/10
 Project_data/val ; batch size = 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/23 [==>...........................] - ETA: 1:23 - loss: 3.4588 - categorical_accuracy: 0.2222

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 76s 3s/step - loss: 2.0295 - categorical_accuracy: 0.2419 - val_loss: 1.5056 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-2.05013-0.23831-1.50562-0.28000.h5
Epoch 2/10
23/23 [==============================] - 6s 278ms/step - loss: 1.6692 - categorical_accuracy: 0.2319 - val_loss: 1.5461 - val_categorical_accuracy: 0.2250

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.66922-0.23188-1.54607-0.22500.h5
Epoch 3/10
23/23 [==============================] - 8s 362ms/step - loss: 1.6144 - categorical_accuracy: 0.2319 - val_loss: 1.5364 - val_categorical_accuracy: 0.2250

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.61439-0.23188-1.53637-0.22500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
23/23 [==============================] - 7s 313ms/step - loss: 1.6354 - categoric

### Experiment:6 on epoch size=30 and batch size =10

In [44]:
batch_size = 10
num_epochs = 30

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  Project_data/train10
Epoch 1/30
 ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 27s 1s/step - loss: 1.4866 - categorical_accuracy: 0.3087 - val_loss: 1.4144 - val_categorical_accuracy: 0.5000

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.48663-0.30870-1.41439-0.50000.h5
Epoch 2/30
23/23 [==============================] - 24s 1s/step - loss: 1.4148 - categorical_accuracy: 0.3739 - val_loss: 1.3727 - val_categorical_accuracy: 0.5750

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.41480-0.37391-1.37272-0.57500.h5
Epoch 3/30
19/23 [=======================>......] - ETA: 4s - loss: 1.3578 - categorical_accuracy: 0.3789

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 23s 1s/step - loss: 1.3763 - categorical_accuracy: 0.3610 - val_loss: 1.3543 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.37419-0.36364-1.35434-0.55000.h5
Epoch 4/30
23/23 [==============================] - 8s 368ms/step - loss: 1.3962 - categorical_accuracy: 0.3913 - val_loss: 1.4192 - val_categorical_accuracy: 0.4000

Epoch 00004: saving model to model_init_2020-06-2907_51_26.335314/model-00004-1.39623-0.39130-1.41917-0.40000.h5
Epoch 5/30
23/23 [==============================] - 8s 367ms/step - loss: 1.4777 - categorical_accuracy: 0.2899 - val_loss: 1.3000 - val_categorical_accuracy: 0.5250

Epoch 00005: saving model to model_init_2020-06-2907_51_26.335314/model-00005-1.47773-0.28986-1.29998-0.52500.h5
Epoch 6/30
23/23 [==============================] - 8s 339ms/step - loss: 1.4267 - categorical_accuracy: 0.3188 - val_loss: 1.2962 - val_categorical_accuracy: 0.4750

Epoch 

### Experiment:7 on epoch size=40 and batch size =10

In [45]:
batch_size = 10
num_epochs = 40

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 25s 1s/step - loss: 1.3015 - categorical_accuracy: 0.4261 - val_loss: 1.2972 - val_categorical_accuracy: 0.4500

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.30151-0.42609-1.29717-0.45000.h5
Epoch 2/40
23/23 [==============================] - 27s 1s/step - loss: 1.2997 - categorical_accuracy: 0.4130 - val_loss: 1.2160 - val_categorical_accuracy: 0.5750

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.29974-0.41304-1.21603-0.57500.h5
Epoch 3/40
19/23 [=======================>......] - ETA: 4s - loss: 1.2886 - categorical_accuracy: 0.4421

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 23s 998ms/step - loss: 1.2728 - categorical_accuracy: 0.4547 - val_loss: 1.2323 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.28037-0.44498-1.23227-0.55000.h5
Epoch 4/40
23/23 [==============================] - 8s 329ms/step - loss: 1.3739 - categorical_accuracy: 0.4058 - val_loss: 1.2082 - val_categorical_accuracy: 0.5000

Epoch 00004: saving model to model_init_2020-06-2907_51_26.335314/model-00004-1.37389-0.40580-1.20818-0.50000.h5
Epoch 5/40
23/23 [==============================] - 7s 319ms/step - loss: 1.3536 - categorical_accuracy: 0.3623 - val_loss: 1.2465 - val_categorical_accuracy: 0.5250

Epoch 00005: saving model to model_init_2020-06-2907_51_26.335314/model-00005-1.35356-0.36232-1.24651-0.52500.h5
Epoch 6/40
23/23 [==============================] - 8s 349ms/step - loss: 1.4030 - categorical_accuracy: 0.3333 - val_loss: 1.1796 - val_categorical_accuracy: 0.6000

Epo

### Experiment:8 on epoch size=25 and batch size =10

In [46]:
batch_size = 10
num_epochs = 25

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path = Epoch 1/25
 Project_data/train ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 26s 1s/step - loss: 1.3443 - categorical_accuracy: 0.4000 - val_loss: 1.3346 - val_categorical_accuracy: 0.4000

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.34432-0.40000-1.33459-0.40000.h5
Epoch 2/25
23/23 [==============================] - 26s 1s/step - loss: 1.3011 - categorical_accuracy: 0.4435 - val_loss: 1.2350 - val_categorical_accuracy: 0.5750

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.30114-0.44348-1.23504-0.57500.h5
Epoch 3/25
19/23 [=======================>......] - ETA: 4s - loss: 1.2924 - categorical_accuracy: 0.4316

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 23s 1s/step - loss: 1.3181 - categorical_accuracy: 0.4124 - val_loss: 1.1760 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.30223-0.43062-1.17605-0.50000.h5
Epoch 4/25
23/23 [==============================] - 7s 311ms/step - loss: 1.3421 - categorical_accuracy: 0.4203 - val_loss: 1.3281 - val_categorical_accuracy: 0.4750

Epoch 00004: saving model to model_init_2020-06-2907_51_26.335314/model-00004-1.34206-0.42029-1.32809-0.47500.h5
Epoch 5/25
23/23 [==============================] - 9s 396ms/step - loss: 1.4552 - categorical_accuracy: 0.3043 - val_loss: 1.1410 - val_categorical_accuracy: 0.6500

Epoch 00005: saving model to model_init_2020-06-2907_51_26.335314/model-00005-1.45516-0.30435-1.14096-0.65000.h5
Epoch 6/25
23/23 [==============================] - 9s 401ms/step - loss: 1.2955 - categorical_accuracy: 0.4928 - val_loss: 1.3146 - val_categorical_accuracy: 0.4000

Epoch 

### Experiment:9 on epoch size=25 and batch size =20

In [47]:
batch_size = 20
num_epochs = 25

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train ; batch size = 20
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 55s 2s/step - loss: 1.2840 - categorical_accuracy: 0.4217 - val_loss: 1.2465 - val_categorical_accuracy: 0.5000

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.28402-0.42174-1.24645-0.50000.h5
Epoch 2/25
 9/23 [==========>...................] - ETA: 29s - loss: 1.2717 - categorical_accuracy: 0.4444

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 27s 1s/step - loss: 1.3618 - categorical_accuracy: 0.3815 - val_loss: 1.2661 - val_categorical_accuracy: 0.5125

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.29685-0.42259-1.26612-0.51250.h5
Epoch 3/25
23/23 [==============================] - 8s 362ms/step - loss: 1.3270 - categorical_accuracy: 0.4058 - val_loss: 1.2325 - val_categorical_accuracy: 0.5375

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.32704-0.40580-1.23254-0.53750.h5
Epoch 4/25
23/23 [==============================] - 10s 449ms/step - loss: 1.4153 - categorical_accuracy: 0.3768 - val_loss: 1.2564 - val_categorical_accuracy: 0.5125

Epoch 00004: saving model to model_init_2020-06-2907_51_26.335314/model-00004-1.41531-0.37681-1.25637-0.51250.h5
Epoch 5/25
23/23 [==============================] - 9s 388ms/step - loss: 1.2772 - categorical_accuracy: 0.4638 - val_loss: 1.2133 - val_categorical_accuracy: 0.5375

Epoch

### Experiment:10 on epoch size=30 and batch size =5

In [48]:
batch_size = 5
num_epochs = 30

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  5
Project_data/train ; batch size = 5
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 13s 570ms/step - loss: 1.4206 - categorical_accuracy: 0.4435 - val_loss: 1.3763 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.42064-0.44348-1.37626-0.30000.h5
Epoch 2/30
23/23 [==============================] - 12s 522ms/step - loss: 1.3523 - categorical_accuracy: 0.3826 - val_loss: 1.0579 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.35233-0.38261-1.05790-0.70000.h5
Epoch 3/30
23/23 [==============================] - 14s 626ms/step - loss: 1.3004 - categorical_accuracy: 0.4261 - val_loss: 1.2537 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.30042-0.42609-1.25372-0.50000.h5
Epoch 4/30
23/23 [==============================] - 13s 566ms/step - loss: 1.3555 - categorical_accuracy: 0.3826 - val_loss: 1.2102 - val_categorical_accuracy: 0.6000



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 12s 501ms/step - loss: 1.3370 - categorical_accuracy: 0.4114 - val_loss: 1.2418 - val_categorical_accuracy: 0.5000

Epoch 00006: saving model to model_init_2020-06-2907_51_26.335314/model-00006-1.33114-0.40777-1.24177-0.50000.h5

Epoch 00006: ReduceLROnPlateau reducing learning rate to 7.105427695090692e-18.
Epoch 7/30
23/23 [==============================] - 8s 340ms/step - loss: 1.4135 - categorical_accuracy: 0.2754 - val_loss: 1.2772 - val_categorical_accuracy: 0.7000

Epoch 00007: saving model to model_init_2020-06-2907_51_26.335314/model-00007-1.41349-0.27536-1.27722-0.70000.h5
Epoch 8/30
23/23 [==============================] - 7s 305ms/step - loss: 1.3371 - categorical_accuracy: 0.3768 - val_loss: 1.3558 - val_categorical_accuracy: 0.2500

Epoch 00008: saving model to model_init_2020-06-2907_51_26.335314/model-00008-1.33714-0.37681-1.35580-0.25000.h5

Epoch 00008: ReduceLROnPlateau reducing learning rate to 3.552713847545346e-18.
Epoch 9/

### Experiment:11 on epoch size=25 and batch size =5

In [49]:
batch_size = 5
num_epochs = 25

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 5
Source path =  Project_data/train ; batch size =Epoch 1/25 5



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 13s 572ms/step - loss: 1.2027 - categorical_accuracy: 0.5043 - val_loss: 1.1792 - val_categorical_accuracy: 0.6000

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.20275-0.50435-1.17921-0.60000.h5
Epoch 2/25
23/23 [==============================] - 12s 520ms/step - loss: 1.3719 - categorical_accuracy: 0.4435 - val_loss: 1.1913 - val_categorical_accuracy: 0.5500

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.37192-0.44348-1.19130-0.55000.h5
Epoch 3/25
23/23 [==============================] - 14s 621ms/step - loss: 1.4198 - categorical_accuracy: 0.3304 - val_loss: 1.2794 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.41983-0.33043-1.27936-0.50000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.734723558371751e-21.
Epoch 4/25
23/23 [==============================] - 12s 532ms/step - loss: 1.2945 - cat

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 12s 507ms/step - loss: 1.3219 - categorical_accuracy: 0.4114 - val_loss: 1.2583 - val_categorical_accuracy: 0.4500

Epoch 00006: saving model to model_init_2020-06-2907_51_26.335314/model-00006-1.32230-0.40777-1.25833-0.45000.h5
Epoch 7/25
23/23 [==============================] - 8s 359ms/step - loss: 1.4376 - categorical_accuracy: 0.4058 - val_loss: 1.2826 - val_categorical_accuracy: 0.5000

Epoch 00007: saving model to model_init_2020-06-2907_51_26.335314/model-00007-1.43757-0.40580-1.28263-0.50000.h5

Epoch 00007: ReduceLROnPlateau reducing learning rate to 4.336808895929377e-22.
Epoch 8/25
23/23 [==============================] - 8s 333ms/step - loss: 1.3482 - categorical_accuracy: 0.3333 - val_loss: 1.1251 - val_categorical_accuracy: 0.6500

Epoch 00008: saving model to model_init_2020-06-2907_51_26.335314/model-00008-1.34819-0.33333-1.12508-0.65000.h5
Epoch 9/25
23/23 [==============================] - 7s 325ms/step - loss: 1.3983 - catego

### Experiment:12 on epoch size=24 and batch size =10

In [ ]:
batch_size = 10
num_epochs = 24

train_generator = generator_new(train_path, train_doc, batch_size)
val_generator = generator_new(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1


model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train Project_data/val ; batch size = ; batch size = 10
Epoch 1/24
 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.3178 - categorical_accuracy: 0.4108

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 74s 1s/step - loss: 1.3147 - categorical_accuracy: 0.4095 - val_loss: 1.2419 - val_categorical_accuracy: 0.5200

Epoch 00001: saving model to model_init_2020-06-2907_51_26.335314/model-00001-1.31541-0.41026-1.24190-0.52000.h5
Epoch 2/24
67/67 [==============================] - 24s 363ms/step - loss: 1.3505 - categorical_accuracy: 0.4030 - val_loss: 1.2416 - val_categorical_accuracy: 0.5200

Epoch 00002: saving model to model_init_2020-06-2907_51_26.335314/model-00002-1.35052-0.40299-1.24156-0.52000.h5
Epoch 3/24
67/67 [==============================] - 23s 344ms/step - loss: 1.3189 - categorical_accuracy: 0.4229 - val_loss: 1.2434 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2020-06-2907_51_26.335314/model-00003-1.31889-0.42289-1.24337-0.52000.h5
Epoch 4/24
67/67 [==============================] - 23s 339ms/step - loss: 1.3538 - categorical_accuracy: 0.4279 - val_loss: 1.2412 - val_categorical_accuracy: 0.5200

Epo

# Observation On Different Experiments:

### Experiment:1 on Batch Size =20
categorical_accuracy: 0.2255 - val_loss: 1.6670 - val_categorical_accuracy: 0.2100

### Experiment:2 on Batch Size =30
loss: 2.1284 - categorical_accuracy: 0.1014 - val_loss: 1.5699 - val_categorical_accuracy: 0.2500

### Experiment:3 on batch size =40
loss: 12.4974 - categorical_accuracy: 0.2215 - val_loss: 12.6258 - val_categorical_accuracy: 0.2167

### Experiment:4 on epoch size=20 and batch size =40
categorical_accuracy: 0.1869 - val_loss: 13.1631 - val_categorical_accuracy: 0.1833

### Experiment:5 on epoch size=15 and batch size =30
categorical_accuracy: 0.3913 - val_loss: 1.4468 - val_categorical_accuracy: 0.4250

### Experiment:6 on epoch size=30 and batch size =10
loss: 1.2786 - categorical_accuracy: 0.4348 - val_loss: 1.2393 - val_categorical_accuracy: 0.6000

### Experiment:7 on epoch size=40 and batch size =10

loss: 1.3714 - categorical_accuracy: 0.3768 - val_loss: 1.2383 - val_categorical_accuracy: 0.5750

### Experiment:8 on epoch size=25 and batch size =10
categorical_accuracy: 0.5362 - val_loss: 1.2063 - val_categorical_accuracy: 0.5000


### Experiment:9 on epoch size=25 and batch size =20
loss: 1.3805 - categorical_accuracy: 0.3333 - val_loss: 1.2503 - val_categorical_accuracy: 0.5250


### Experiment:10 on epoch size=30 and batch size =5
loss: 1.2232 - categorical_accuracy: 0.4783 - val_loss: 1.3189 - val_categorical_accuracy: 0.4500

### Experiment:11 on epoch size=25 and batch size =10
loss: 1.3390 - categorical_accuracy: 0.8258 - val_loss: 1.2182 - val_categorical_accuracy: 0.78.59

### Experiment:12 on epoch size=24 and batch size =10
loss: 1.3689 - categorical_accuracy: 0.4378 - val_loss: 1.2399 - val_categorical_accuracy: 0.5200



## The best experiment gives categorical_accuracy 82.58 % and validation accuracy 78.59 %